### Obtaining Data

#### Yahoo Finance and Pandas

In [ ]:
from __future__ import print_function
import datetime
import pandas.io.data as web

if __name__ == "__main__":
    spy = web.DataReader("SPY", "yahoo",
                         datetime.datetime(2007,1,1),
                         datetime.datetime(2015,6,15))
print(spy.tail())

In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
# quandl_data.py


from __future__ import print_function
import matplotlib.pyplot as plt
import pandas as pd
import requests

def construct_futures_symbols(symbol, start_year=2010, end_year=2014):
    """
    Constructs a list of futures contract codes
    for a particular symbol and timeframe.
    """
    
    futures = []
    # March, June, September and
    # December delivery codes
    months = 'HMUZ'
    for y in range(start_year, end_year+1):
        for m in months:
            futures.append("%s%s%s" % (symbol, m, y))
            
    return futures


def download_contract_from_quandl(contract, dl_dir):
    """
    Download an individual futures contract from Quandl and then
    store it to disk in the 'dl_dir' directory. An auth_token is
    required, which is obtained from the Quandl upon sign-up.
    """
    
    # Construct the API call from the contract and auth_token
    api_call = "http://www.quandl.com/api/v1/datasets/"
    api_call += "OFDP/FUTURE_%s.csv" % contract
    # If you wish to add an auth token for more downloads, simply
    # comment the following line and replace MY_AUTH_TOKEN with
    # your auth token in the line below
    params = "?sort_order=asc"
    #params = "?auth_token=MY_AUTH_TOKEN&sort_order=asc"
    full_url = "%s%s" % (api_call, params)
    
    # Download the data from Quandl
    data = requests.get(full_url).text
    
    # Store the data to disk
    fc = open('%s/%s.csv' % (dl_dir, contract), 'w')
    fc.write(data)
    fc.close()


def download_historical_contracts(symbol, dl_dir, 
                                  start_year=2010, 
                                  end_year=2014):
    """
    Downloads all futures contracts for a specified symbol
    between a start_year and an end_year.
    """

    contracts = construct_futures_symbols(symbol, start_year, end_year)

    for c in contracts:
        print("Downloading contract: %s" % c)
        download_contract_from_quandl(c, dl_dir)
        
if __name__ == "__main__":
    symbol = 'ES'
    
    # Make sure you’ve created this
    # relative directory beforehand
    dl_dir = 'quandl/futures/ES'
    
    # Create the start and end years
    start_year = 2010
    end_year = 2014
    
    # Download the contracts into the directory
    download_historical_contracts(symbol, 
                                  dl_dir, 
                                  start_year, 
                                  end_year)
    
    # Open up a single contract via read_csv
    # and plot the settle price
    es = pd.io.parsers.read_csv("%s/ESH2010.csv" % dl_dir, index_col="Date")
    es["Settle"].plot()
    plt.show()